# INDEX
1. [Feature Engineering & Customer Insights](#feature-engineering--customer-insights)
2. [Tools Used](#tools-used)
4. [Architecture Overview](#architecture-overview-custom-modular-layer-based-architecture)
5. [Exploratory Data Analysis (EDA)](#exploratory-data-analysis-eda)
6. [Data Cleaning](#data-cleaning)
    - Duplicate Checks
    - 🕓 Timestamp Nulls and Fulfillment Gaps
    - Data loss
7. [Data Quality](#data-quality)
    - Data Tests
    - Source Freshness Checks
8. [Sanity Checks](#sanity-checks)
    - Sanity Check UnitPrice
    - Sanity Check Quantity
    - Sanity Check Product pricing consistency
9. [Business Logic Exclusions](#business-logic-exclusions)
10. [Data Transformation](#data-transformation)
    - Macro Function
    - How KPIS & models were built
11. [Business Intelligence Dashboard](#business-intelligence-dashboard)
12. [Data Modeling](#data-modeling)
    - OBT (One big table)
    - Star Schema
    - Incremental models
13. [Data Lineage](#data-lineage)
14. [Data Orchestration](#data-orchestration)
    - Cron jobs
    - Partitioning
15. [CI/CD](#ci-cd)
    - Dockerfile
    - Github Actions
    - GCP Artifact Registery
16. [Conclusion](#conclusion)





## 🧩 Problem Statement

This project transforms raw Point-of-Sale (POS) transaction logs into actionable customer intelligence by engineering and segmenting key behavioral and financial metrics. The pipeline performs the following computations and classifications:

### Feature Engineering & Customer Insights

* **Average Order Value (AOV)**

  * Per-customer AOV
  * Overall AOV across all customers
* **Customer Lifetime Value (CLTV)**

  * Individual CLTV based on historical purchases and frequency
  * Overall CLTV across the customer base
  * CLTV-based segmentation (e.g. High, Medium, Low value)

* **RFM Analysis**

  * Computed Recency, Frequency, and Monetary metrics per customer
  * RFM segmentation (e.g. champions, at risk, hibernating)
* **Churn Prediction**

  * Flag churned customers based on inactivity beyond a defined purchase window
  * Churn Risk segmentation (e.g. Low, Medium, High Risk)

* **Engagement Metrics**

  * Daily Active Users (DAU)
  * Monthly Active Users (MAU)
  * Stickiness Ratio (DAU/MAU)

### ✅ Input Table (Truncated for Readability)
| INVOICENO | STOCKCODE | DESCRIPTION                           | QUANTITY | INVOICEDATE           | UNITPRICE | CUSTOMERID | COUNTRY         |
|-----------|-----------|---------------------------------------|----------|------------------------|-----------|------------|-----------------|
| 536365    | 85123A    | WHITE HANGING HEART T-LIGHT HOLDER    | 6        | 2010-12-01 08:26:00   | 2.55      | 17850.0    | United Kingdom  |
| 536365    | 71053     | WHITE METAL LANTERN                   | 6        | 2010-12-01 08:26:00   | 3.39      | 17850.0    | United Kingdom  |
| 536365    | 84406B    | CREAM CUPID HEARTS COAT HANGER        | 8        | 2010-12-01 08:26:00   | 2.75      | 17850.0    | United Kingdom  |
| ...       | ...       | ...                                   | ...      | ...                    | ...       | ...        | ...             |


### ✅ Output Table (Customer Summary – Part 1)

| CUSTOMERID | RECENCY | MONETARY | FREQUENCY | TENURE | R\_SCORE | F\_SCORE | M\_SCORE |
| ---------- | ------- | -------- | --------- | ------ | -------- | -------- | -------- |
| 17850      | 303     | 5288.63  | 35        | 374    | 5        | 5        | 5        |

➡️ *Continued below to fit on one page*

---

### ✅ Output Table (Customer Summary – Part 2)

| RFM\_SEGMENTATION | CUSTOMER\_CLTV\_PROFIT | CLTV\_PROFIT\_SEGMENTATION | ISCHURNED | CHURN\_RISK\_GROUP |
| ----------------- | ---------------------- | -------------------------- | --------- | ------------------ |
| Champions / VIPs  | 133131.089811720       | High                       | true      | Churned and input  |



### 🎯 Why This Matters

This solution supports data-driven decision-making in customer relationship management, by enabling:

* **Customer Prioritization**: Identify and focus on high-value and at-risk segments
* **Personalized Marketing**: Enable tailored campaigns based on behavior and value
* **Retention Strategy**: Detect churn patterns early to re-engage slipping customers
* **Revenue Optimization**: Adjust offers, pricing, and promotions based on customer metrics
* **Executive Insights**: Provide KPIs for leadership to monitor customer health over time
* **Product Feedback Loops**: Align marketing and product decisions with customer behavior trends

---

## Tools Used

* **Python** – Utilized for rapid prototyping, data wrangling, feature engineering, and initial transformation logic prior to dbt implementation
* **SQL** – Employed for exploratory analysis, ad hoc queries, and core transformations within dbt models
* **dbt (Data Build Tool)** – Used for scalable data transformations, implementing modular data models, enforcing data quality tests, CI/CD integration, and generating automated documentation
* **Snowflake** – Cloud data warehouse for centralized storage and compute, enabling scalable querying and analytics
* **Plotly** – Interactive visualization library used for generating rich, responsive charts and exploratory visual analysis
* **Streamlit** – Lightweight web framework for building the final interactive dashboard, enabling end-user engagement with KPIs and segmentation outputs

---

## Architecture Overview: Custom modular layer-based architecture 

## 🏗️ Architecture Overview: Modular Layered Data Modeling with dbt

To enable reliable, scalable, and transparent transformations, I designed and implemented a **custom modular layer-based architecture** using **dbt**, tailored specifically for eCommerce delivery and customer analytics.

This architecture follows the **staging → intermediate → mart** pattern, ensuring clean separation of concerns, high reusability, and clear data lineage across the pipeline.

---

### 🔹 Layer 1: **Staging (`stg_`)**

This layer standardizes raw POS transaction data by cleaning, renaming, and typing columns.

* **`stg_transaction_logs.sql`**:
  Raw transaction data is ingested and standardized here before transformation logic is applied.

---

### 🔹 Layer 2: **Intermediate (`int_`)**

This layer performs reusable business logic transformations, aggregations, and metric computations.

**Key intermediate models:**

* `int_customer_aov.sql` – Computes per-customer Average Order Value
* `int_customer_cltv.sql` – Derives Customer Lifetime Value metrics
* `int_customer_cltv_segmentation.sql` – Segments customers by CLTV tiers
* `int_customer_rfm_score.sql` – Calculates Recency, Frequency, and Monetary scores
* `int_customer_churn_flag.sql` – Flags customers as churned based on purchasing windows
* `int_customer_churn_segmentation.sql` – Classifies churn risk segments
* `int_customer_dau_mau_stickiness.sql` – Calculates engagement metrics (DAU, MAU, Stickiness)

These intermediate models are **modular** and **refactored**, allowing downstream mart models to pull exactly what they need with minimal coupling.

---

### 🔹 Layer 3: **Mart (`obt_`)**

This layer creates analytics-ready tables for dashboards and reporting, often joined from multiple intermediate models.

**Final data marts:**

* `obt_customer_kpis_and_segmentation`

  * Joins customer-level metrics from all intermediate models into one wide table
  * Includes AOV, CLTV, RFM scores, churn flags, segmentation labels, and engagement metrics
* `obt_overall_customer_kpis`

  * Aggregates overall metrics across all customers (e.g., average CLTV, total DAUs)

---

### 🔄 Joins & Relationships

Each mart model is built using `ref()` to pull cleanly from the intermediate layer:

* `obt_customer_kpis_and_segmentation` joins on `customer_id` across:

  * `int_customer_aov`
  * `int_customer_cltv`
  * `int_customer_rfm_score`
  * `int_customer_churn_flag`
  * `int_customer_cltv_segmentation`
  * `int_customer_churn_segmentation`
  * `int_customer_dau_mau_stickiness`


---

## ✅ Why This Approach Works
This structure supports **lineage tracking**, **test coverage**, and **CI/CD integration**, ensuring data trustworthiness from raw logs to dashboard-ready insights.



---

##  Exploratory Data Analysis (EDA)

* Inspected column names, data types, and overall schema for consistency
* Verified completeness by checking for nulls, anomalies, and malformed records
* Counted unique values per column to identify categorical features, keys, and potential grouping fields

---

##  Data Cleaning

I performed systematic cleaning across all BRONZE tables to prepare the data for downstream merging and analysis.

---

### 📂 Source: Layer 1: **Staging (`stg_`)

**Key Cleaning Actions:**

---

### 🔍 Duplicate Checks

* **No duplicates** found in stg_transaction_logs (raw transaction logs)


---

### 🕓 Timestamp Nulls and Fulfillment Gaps

**Columns affected by nulls:**

| Column Name   | Null Count | Most Likely Reason (When Both Are Null)                                                                                                                                                                                                                                                                                                                                                          |
| ------------- | ---------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| `Description` | 1,454      | When `Description` and `CustomerID` are both null, and `UnitPrice = 0.0`, these rows most likely represent **system-generated inventory adjustments** or **manual returns** where no product was properly recorded and no customer was involved. If `Quantity` is **negative**, it's likely a **write-off for lost/damaged stock**. If **positive**, it may reflect a **restock or test entry**. |
| `CustomerID`  | 135,080    | For the subset where `CustomerID` and `Description` are both null, these are likely **non-customer-facing transactions** — such as inventory corrections, returns without proper logging, or zero-value backend entries — not intended to track customer behavior or product sales. If only CUstomerID null the purchase was made without a registered customer account — likely a guest checkout. or if StockCode = DOT then non-product related item such as Postage costs.                                                                                                              |
# Sanity Checks
## 1. Sanity Check UnitPrice
#### 1.1 Negative `UnitPrice` flagged (debt write-offs)
EXAMPLE 
	InvoiceNo	StockCode	Description	Quantity	InvoiceDate	UnitPrice	CustomerID	Country
299983	A563186	B	Adjust bad debt	1	2011-08-12 14:51:00	-11062.06	NaN	United Kingdom
299984	A563187	B	Adjust bad debt	1	2011-08-12 14:52:00	-11062.06	NaN	United Kingdom

#### 1.2 Unit Price != 0
| Group Label                          | Keywords Trigger                                                               |
| ------------------------------------ | ------------------------------------------------------------------------------ |
| `Damaged or Written-Off`             | `damaged`, `damages`, `returned`, `given away`, `broken`                       |
| `Inventory Adjustment`               | `adjustment`, `taig adjust`, `manual`, `website fixed`, `stock check`, `check` |
| `Logistics (Dotcom / Amazon / eBay)` | `dotcom`, `amazon`, `ebay`, `mailout`                                          |
| `Coding / Mapping Issues`            | `wrongly coded`, `incorrectly credited`, `Actual description`                  |
| `Unknown / Incomplete`               | `??`, `missing`, `test`, `manual`, `?display?`                                 |
| `Bundling / Set Handling`            | `sold as set`, `sold as sets`, `dotcom sold sets`, `amazon sold sets`          |
| `Promotional / Internal Use`         | `showroom`, `display`, `samples`, `given away`                                 |


## 2. Sanity Check Quantity

🔄 Why `Quantity` Is Negative: Business Logic Behind Returns & Adjustments

In this e-commerce dataset, a **negative `Quantity`** typically indicates a **reverse transaction**, such as a **return, refund, discount, or stock correction**. These are not new sales — they are credits or adjustments against previous sales.

Below are the key scenarios that explain why quantity is `< 0`:

✅  **I. Invoice Number Starts with 'C'**
**Example**: `C2773`
* This prefix conventionally stands for **Credit Note** — a reversal of a previous invoice.
* It indicates the customer was refunded or the sale was cancelled.
* Negative quantity reflects the product being removed from revenue and potentially restocked.

✅ **II. StockCode Contains 'C'**
**Example**: `C35004C`
* These StockCodes often represent **returns**.
* The "C" may be used internally to flag **credit items**.
* Quantity is negative because it's offsetting a prior positive sale.

✅ **III. StockCode = 'D' and Description = 'Discount'**
* These rows represent **line-item discounts** applied to orders.
* The `Quantity` is set to `-1` (or another negative number) as a way of **applying a price reduction** in the sales log.

✅ **IV. StockCode = 'M' and Description = 'Manual'**
* These are **manual price corrections** entered by staff.
* Often used to fix pricing errors or override a transaction.
* Negative quantity indicates a **reverse entry or credit** to offset the mistake.

### 🧾 Summary Table of reasons why quantity is `< 0`

| Condition                                        | Interpretation                       |
| ------------------------------------------------ | ------------------------------------ |
| `InvoiceNo` starts with `"C"`                    | Credit note (full or partial refund) |
| `StockCode` contains `"C"`                       | Return    |
| `StockCode = "D"` and `Description = "Discount"` | Discount applied as a negative item  |
| `StockCode = "M"` and `Description = "Manual"`   | Manual price correction by staff     |

---
## 3. Sanity Check Product pricing consistency

To ensure data quality and reliable revenue analysis, I conducted a sanity check on product pricing consistency over time. Abrupt or erratic pricing behavior can signal either legitimate business logic (e.g. promotions) or data quality issues (e.g. manual errors, misentries). This check helps distinguish between the two.

| **Symptom**              | **Date Behavior Pattern**                                                                                                                                                        |
| ------------------------ | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Catalogue Revision**   | Price changes **once** and remains stable afterward. A clear “before” and “after” period, indicating a formal catalogue update.                                                  |
| **Promotion / Discount** | Temporary price drops that occur in **short bursts** (e.g., during Black Friday, seasonal sales). Prices return to normal afterward.                                             |
| **Data Entry Error**     | A single, **isolated price anomaly** with no similar entries nearby. Usually occurs on a stray date and may reflect a manual input mistake.                                      |
| **Zero Price / Manual**  | Scattered zero- or unusually low prices, often linked to **credit notes** (InvoiceNo starts with `"C"`) or **manual adjustments**. These often co-occur with returns or refunds. |

To ensure pricing integrity across products, each SKU (Stock Keeping Unit) was assigned a flag based on how its `UnitPrice` behaved over time. This helped differentiate stable products from those affected by promotions, adjustments, or data issues.

| **Flag Value**                   | **Count**    | **Definition / Pattern in Data**                                                                                     | **Typical Business Cause**                                                | **Recommended Treatment**                                                                                                       |
| -------------------------------- | ------------ | -------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------- |
| **`normal`**                     | *All others* | SKU is consistently sold at a **single non-zero price** throughout the dataset.                                      | Standard catalogue pricing with no fluctuation.                           | Use as-is in all metrics and models (AOV, CLTV, pricing forecasts).                                                             |
| **`oscillating_or_promotional`** | **1,046**    | SKU has **two or more recurring prices** — usually a base price with intermittent discounts (e.g., seasonal sales).  | Time-bound promotions, temporary discounts, or customer-specific pricing. | Retain all prices. For demand modeling, tag discounted rows with a `"promo"` feature to isolate elastic demand behavior.        |
| **`zero_or_manual`**             | **1,034**    | `UnitPrice = 0` or `StockCode = 'M'` / `Description = 'Manual'`. Often isolated and paired with `Quantity = 1`.      | Free samples, goodwill gestures, credit note balancing items.             | Exclude from price analyses; include in net revenue and refund logic (value = £0).                                              |
| **`one_off_price_change`**       | **425**      | Exactly **two distinct prices**, with a **clear, non-overlapping date split**. Example: £6.35 until Jan, then £1.95. | Permanent price revision due to catalogue update or product change.       | Split into pricing eras or retain the later price if forecasting forward-looking demand.                                        |
| **`complex`**                    | **138**      | Irregular pricing: **more than two prices**, overlapping timeframes, and combinations of zero, promos, or outliers.  | Combination of data entry errors, phased pricing, or product repackaging. | Manually inspect. For automation, remove obvious typos, impute modal price, and flag for review or exclude from pricing models. |

---
# Data loss after cleaning is Data loss: 26.54%




##  Data Quality
1. Data Tests
I implemented multiple layers of testing to ensure accuracy and reliability of models:
- Built-in dbt tests → unique, not_null, accepted_values.
- dbt_utils package → extended SQL assertions such as data range checks and conditional validations.

2. Source Freshness Checks
- Validates that source tables are up-to-date and not stale.
- Ensures downstream models are always built on the latest available data.


#  Data Transformation 

## 🔧 Data Transformation – `stg_transaction_logs`

The `stg_transaction_logs` model standardizes raw transactional data to prepare it for accurate customer analytics. All transformations in this stage focus on **cleaning**, **filtering**, and **structuring** the data before it flows into intermediate and mart layers.

---


---

### ✅ What Was Transformed

| Transformation                  | Description                                                                                                                                                        |
| ------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **Business Logic Exclusions**   | Removed invalid transactions (e.g. free samples, returns, manual adjustments, debt, damaged or lost products postage) using clear rules (see [Business Logic Exclusions](#)).                     |
| **Country Mapping & Filtering** | - Excluded `European Community` and `Unspecified` entries. <br> - Standardized country names using a **dbt seed file** to map raw values like `EIRE` to `Ireland`, `RSA` to `South Africa`, `USA` to `United States` and `Channel Islands` to `United Kingdom`. |
| **Data Type Casting**           | Cast all columns to consistent and analysis-ready formats (e.g. `InvoiceNo` as `varchar`, `UnitPrice` as `numeric(12,2)`, `InvoiceDate` as `timestamp`).           |

---

### 🎯 Why These Transformations Were Necessary

* **Ensure analytical integrity**: Removes noise and non-customer behavior from modeling (e.g., CLTV, churn).
* **Support regional analysis**: Clean, consistent country names enable geographic segmentation and filtering.
* **Enforce schema consistency**: Proper data types improve query performance, downstream joins, and dashboard compatibility.

---

### 🛠️ How It Was Done

#### 1. **Business Logic Filtering**

Excluded rows using SQL logic in the dbt model:

```sql
WHERE UnitPrice > 0
  AND Quantity > 0
  AND Description IS NOT NULL
  AND NOT Description ILIKE ANY (ARRAY['%damaged%', '%adjust%', '%manual%', ...])
```
### 📦 1. **Zero-Value Transactions (Free or Display Items)**

**Condition:**

* `UnitPrice = 0`
* `Quantity >= 0`
* `CustomerID IS NOT NULL`
* `InvoiceNo NOT LIKE 'C%'`
* `Description` does **not** contain:
  `'adjust'`, `'manual'`, `'check'`, `'Adjust bad debt'`, `'???'`, `'missing'`, `'POST'`, `'DOT'`, `'CRUK'`, `'BANK CHARGES'`
* `StockCode NOT IN ('M', 'C2')`

**Action:**
Excluded from revenue, CLTV, churn, segmentation.

---

### 🔁 2. **Returns & Refunds**

**Condition:**

* `Quantity < 0`
* OR `InvoiceNo LIKE 'C%'`

**Action:**
Included in net revenue; excluded from CLTV and behavioral models.

---

### 🔄 3. **Inventory Adjustments & Manual Ops Corrections**

**Condition:**

* `StockCode IN ('M', 'C2')`
* OR `Description` contains `'adjust'`, `'manual'`, `'check'`
* AND `Description != 'Adjust bad debt'`
* AND not matched to group 2

**Action:**
Excluded from revenue, CLTV, segmentation.

---

### 💸 4. **Bad Debt & Write-Offs**

**Condition:**

* `Description = 'Adjust bad debt'` (case-insensitive exact/fuzzy match)

**Action:**
Excluded from all behavioral models; retained for financial audit.

---

### ❓ 5. **Incomplete or Suspicious Entries**

**Condition:**

* `CustomerID IS NULL AND UnitPrice = 0`
* OR `Description IN ('???', 'missing', 'wrongly sold')`

**Action:**
Flagged or dropped depending on QA policy.

---

### 🛒 6. **Non-Product / Operational Charges**

**Condition:**

* `Description` contains any of:
  `'POST'`, `'DOT'`, `'MAILOUT'`, `'BANK CHARGES'`, `'CRUK'`
* AND not already classified above

**Action:**
Excluded from SKU-level sales, CLTV, segmentation.

#### 2. **Country Mapping via dbt Seed File**

Used a seed file (`country_mapping.csv`) joined into the model to standardize country names:

```sql
-- Sample of the seed file
raw_country         | mapped_country
--------------------|----------------
EIRE                | Ireland
RSA                 | South Africa
Channel Islands     | United Kingdom
USA                 | United States

-- Join in dbt model
LEFT JOIN {{ ref('country_mapping') }} cm
  ON lower(trim(country)) = lower(trim(cm.raw_country))
```

#### 3. **Casting Data Types**

Ensured compatibility and clean schema:

```sql
CAST(InvoiceNo   AS VARCHAR)        AS InvoiceNo,
CAST(StockCode   AS VARCHAR)        AS StockCode,
CAST(Description AS VARCHAR)        AS Description,
CAST(cm.mapped_country AS VARCHAR)  AS Country,
CAST(UnitPrice   AS NUMERIC(12,2))  AS UnitPrice,
CAST(Quantity    AS INT)            AS Quantity,
CAST(CustomerID  AS INT)            AS CustomerID,
CAST(InvoiceDate AS TIMESTAMP)      AS InvoiceDate
```


## 🔧 Data Transformation – `int_total_amount_calc`

The `int_total_amount_calc` model computes transaction-level revenue by calculating the **total amount spent per line item**. This forms the foundational layer for all **CLTV**, **AOV**, and **segmentation metrics** downstream.

---

### ✅ What Was Transformed

| Transformation               | Description                                                                                                             |
| ---------------------------- | ----------------------------------------------------------------------------------------------------------------------- |
| **Total Amount Calculation** | Computed `TotalAmount` as the product of `Quantity * UnitPrice`, representing the gross amount paid for each line item. |
| **Selected Key Columns**     | Retained only relevant columns for revenue and customer-level analysis.                                                 |

---

### 🎯 Why This Transformation Was Necessary

* **Enable monetary aggregation**: The `TotalAmount` field allows us to roll up revenue by customer and invoice.
* **Support downstream KPIs**: This is a required metric for computing AOV, CLTV, RFM, churn, and segmentation models.
* **Reduce row complexity**: By selecting only the required fields, the model remains lean and focused on financial analysis.

---

### 🛠️ How It Was Done

#### 1. **TotalAmount Calculation**

The core transformation is the line-level amount:

```sql
Quantity * UnitPrice AS TotalAmount
```

This accurately reflects what a customer paid for each product unit on each invoice line.

#### 2. **Column Selection**

Only the necessary columns were retained to support clean joins and aggregations:

```sql
SELECT
    CustomerID,
    InvoiceNo,
    InvoiceDate,
    Description,
    Quantity,
    UnitPrice,
    Quantity * UnitPrice AS TotalAmount,
    Country
FROM {{ ref('stg_transaction_logs') }}
```

## 🔧 Data Transformation – `int_customer_aov`

The `int_customer_aov` model calculates the **Average Order Value (AOV)** per customer, based on all their historical transactions. This metric is a key input into customer segmentation ie. CLTV modeling.

---

### ✅ What Was Transformed

| Transformation                 | Description                                                                       |
| ------------------------------ | --------------------------------------------------------------------------------- |
| **Customer-Level Aggregation** | Grouped transactions by `CustomerID` and calculated the average of `TotalAmount`. |
| **AOV Calculation**            | Computed `CustomerAOV` using `AVG(TotalAmount)` per customer.                     |
| **Field Selection**            | Retained only `CustomerID` and `CustomerAOV` for lean, focused output.            |

---

### 🎯 Why This Transformation Was Necessary

* **Measure purchase behavior**: AOV reflects how much, on average, each customer spends per transaction.
* **Enable value-based segmentation**: Helps classify customers into high-, medium-, or low-spenders.
* **Support CLTV modeling**: A core component in lifetime value estimation (LTV = AOV × purchase frequency × lifespan x gross margin).

---

### 🛠️ How It Was Done

#### 1. **AOV Calculation per Customer**

The transformation used a simple aggregation over the transactional data:

```sql
SELECT
    CustomerID,
    SUM(TotalAmount)/COUNT(InvoiceNo) AS CustomerAOV
FROM {{ ref('int_total_amount_calc') }}
GROUP BY CustomerID
```

#### 2. **Selected Fields**

Only the fields needed for downstream joins and modeling were kept:

* `CustomerID`
* `CustomerAOV`

---

## 🔧 Data Transformation – `int_customer_cltv`

The `int_customer_cltv` model calculates **Customer Lifetime Value (CLTV)** for each customer, expressed in both revenue and profit terms. This model combines behavioral metrics (frequency and tenure) with spending patterns (AOV) and business-level profitability (gross margin).

---

### ✅ What Was Transformed

| Transformation                 | Description                                                                                          |
| ------------------------------ | ---------------------------------------------------------------------------------------------------- |
| **CLTV (Revenue) Calculation** | Estimated revenue per customer using: `CustomerAOV × Frequency × Tenure`.                            |
| **CLTV (Profit) Calculation**  | Multiplied CLTV revenue by `gross_margin` to estimate lifetime profit contribution.                  |
| **Field Selection**            | Retained only key identifiers and computed metrics: `CustomerID`, `CLTV (Revenue)`, `CLTV (Profit)`. |

---

### 🎯 Why This Transformation Was Necessary

* **Quantifies customer value**: Allows the business to prioritize high-value customers for marketing and retention.
* **Supports strategic decision-making**: Used for offer targeting, segmentation, pricing, and churn mitigation.
* **Enables profit-based modeling**: Distinguishes between top spenders and top contributors to actual bottom-line profit.

---

### 🛠️ How It Was Done

#### 1. **Inputs Joined**

Joined the following models:

* `a` – from `int_customer_aov`: contains `CustomerAOV`
* `r` – from `int_customer_behavior_metrics`: contains `Frequency` and `Tenure`
* `c` – from `ref('business_constants')` or config: provides `gross_margin`

#### 2. **CLTV Revenue Formula**

```sql
CustomerAOV × Frequency × Tenure AS customer_cltv_revenue
```

#### 3. **CLTV Profit Formula**

```sql
CustomerAOV × Frequency × Tenure × gross_margin AS customer_cltv_profit
```

#### 4. **Final Output**

```sql
SELECT
    r.CustomerID,
    a.CustomerAOV * r.Frequency * r.Tenure AS customer_cltv_revenue,
    a.CustomerAOV * r.Frequency * r.Tenure * c.gross_margin AS customer_cltv_profit
FROM {{ ref('int_customer_behavior_metrics') }} r
JOIN {{ ref('int_customer_aov') }} a ON r.CustomerID = a.CustomerID
CROSS JOIN {{ ref('business_constants') }} c
```
---

## 🔧 Data Transformation – `int_customer_rfm_values`

The `int_customer_rfm_values` model calculates **RFM (Recency, Frequency, Monetary)** metrics for each customer. These are foundational for behavioral segmentation and customer value scoring.

---

### ✅ What Was Transformed

| Metric        | Description                                                                                                        |
| ------------- | ------------------------------------------------------------------------------------------------------------------ |
| **Recency**   | Days since the customer's most recent purchase: `DATEDIFF(day, MAX(InvoiceDate), snapshot_date)`                   |
| **Tenure**    | Days between the customer's first purchase and the snapshot date: `DATEDIFF(day, MIN(InvoiceDate), snapshot_date)` |
| **Frequency** | Total number of distinct purchases: `COUNT(DISTINCT InvoiceNo)`                                                    |
| **Monetary**  | Total amount the customer has spent: `SUM(TotalAmount)`                                                            |

---

### 🎯 Why This Transformation Was Necessary

* **Recency** captures how recently the customer was active — useful for churn detection.
* **Frequency** reflects engagement and loyalty.
* **Monetary** shows how valuable the customer is.
* **Tenure** helps in calculating behavioral lifecycle metrics (e.g., CLTV time horizon).
* Together, RFM is a **proven framework** for customer segmentation, lifecycle tracking, and targeting strategy.

---

### 🛠️ How It Was Done

#### 1. **Join with Snapshot Date**

The model uses a **snapshot date** (from a dbt seed or config) to anchor all time-based calculations:

```sql
SELECT
    CustomerID,
    DATEDIFF(day, MAX(InvoiceDate), s.snapshot_date) AS Recency,
    DATEDIFF(day, MIN(InvoiceDate), s.snapshot_date) AS Tenure,
    COUNT(DISTINCT InvoiceNo) AS Frequency,
    SUM(TotalAmount) AS Monetary
FROM {{ ref('int_total_amount_calc') }} t
CROSS JOIN {{ ref('snapshot_date') }} s
GROUP BY CustomerID, s.snapshot_date
```


## 🔧 Data Transformation – `int_customer_dau_mau_stickiness`

The `int_customer_dau_mau_stickiness` model calculates **engagement metrics** over time by tracking how many unique customers interact with the platform daily (DAU), over a rolling monthly window (MAU), and how "sticky" the product is (Stickiness = DAU / MAU).

---

### ✅ What Was Transformed

| Metric         | Description                                                                                 |
| -------------- | ------------------------------------------------------------------------------------------- |
| **DAU**        | Count of distinct `CustomerID`s per `InvoiceDate` — representing **Daily Active Users**.    |
| **MAU**        | 30-day rolling sum of DAU using a window function — representing **Monthly Active Users**.  |
| **Stickiness** | Ratio of DAU to MAU (`DAU / MAU`), rounded to two decimals — measuring platform stickiness. |

---

### 🎯 Why This Transformation Was Necessary

* **Track engagement over time**: Understand how consistently customers return to shop.
* **Benchmark activity health**: DAU/MAU Stickiness is a key SaaS/ecommerce metric to evaluate retention and product "habit formation".
* **Enable time-based segmentation**: Helps separate loyal customers from one-time buyers.

---

### 🛠️ How It Was Done

#### 1. **DAU Calculation**

Calculated per day:

```sql
COUNT(DISTINCT CustomerID) AS DAU
```

#### 2. **MAU (30-day Rolling Window)**

Computed using a window function:

```sql
SUM(DAU) OVER (
    ORDER BY InvoiceDate
    ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
) AS MAU
```

#### 3. **Stickiness Ratio**

Handled division-by-zero safely:

```sql
ROUND(DAU / NULLIF(MAU, 0), 2) AS Stickiness
```

#### 4. **Selected Columns**

Only core engagement metrics were kept:

```sql
SELECT
    InvoiceDate,
    DAU,
    MAU,
    ROUND(DAU / NULLIF(MAU, 0), 2) AS Stickiness
FROM ...
```
---
## 🔧 Data Transformation – `int_customer_churn_flag`

The `int_customer_churn_flag` model assigns a binary churn label to each customer based on their **Recency** — the number of days since their last transaction — as of a defined **snapshot date**.  

---

### 🛠 Macro Definition

```jinja
{% macro churn_flag(recency_col, churn_window=90) %}
    -- Returns TRUE if the customer is churned (Recency > churn_window), else FALSE.
    ({{ recency_col }} > {{ churn_window }})
{% endmacro %}

```sql
WITH churn_flag_table AS (
    SELECT
        CustomerID,
        {{ churn_flag('Recency', var('churn_window', 90)) }} AS IsChurned
    FROM {{ ref('int_customer_rfm_values') }}
)
SELECT *
FROM churn_flag_table
```


---

## 🔧 Data Transformation – `int_customer_rfm_values`

The `int_customer_rfm_score` model assigns **quantile-based scores (1–5)** to each customer across the three RFM dimensions — **Recency**, **Frequency**, and **Monetary** — enabling behavioral segmentation.

---

### ✅ What Was Transformed

| Transformation      | Description                                                                                              |
| ------------------- | -------------------------------------------------------------------------------------------------------- |
| **Recency Score**   | Customers are ranked into 5 buckets using `ntile(5)` — lower recency = higher score (i.e., more recent). |
| **Frequency Score** | Higher frequency of purchase → higher `f_score`. Scored using `ntile(5)`.                                |
| **Monetary Score**  | Customers who spent more receive higher `m_score`, also via `ntile(5)`.                                  |
| **Field Selection** | Retained only `CustomerID`, `r_score`, `f_score`, and `m_score`.                                         |

---

### 🎯 Why This Transformation Was Necessary

* **Enables RFM Segmentation**: Scoring simplifies raw metrics into interpretable levels of customer value.
* **Supports customer targeting**: Identifies "Champions", "At Risk", "Hibernating", and other lifecycle stages.
* **Enables modeling and visualization**: Scores can be grouped into RFM bands for easier cohort analysis.

---

### 🛠️ How It Was Done

#### 1. **Score Calculation with Window Functions**

Using `ntile(5)` to assign quintile-based ranks:

```sql
SELECT
    CustomerID,
    NTILE(5) OVER (ORDER BY Recency ASC) AS r_score,   -- Lower recency → higher score
    NTILE(5) OVER (ORDER BY Frequency DESC) AS f_score, -- Higher frequency → higher score
    NTILE(5) OVER (ORDER BY Monetary DESC) AS m_score   -- Higher monetary → higher score
FROM {{ ref('int_customer_rfm_values') }}
```

#### 2. **Scoring Logic**

* `r_score`: 5 = most recent, 1 = longest inactive
* `f_score`: 5 = most frequent, 1 = least frequent
* `m_score`: 5 = highest spender, 1 = lowest spender

## 🔧 Data Transformation – `int_customer_rfm_score`

The `int_customer_rfm_score` model assigns **quantile-based scores (1–5)** to each customer across the three RFM dimensions — **Recency**, **Frequency**, and **Monetary** — enabling behavioral segmentation.

---

### ✅ What Was Transformed

| Transformation      | Description                                                                                              |
| ------------------- | -------------------------------------------------------------------------------------------------------- |
| **Recency Score**   | Customers are ranked into 5 buckets using `ntile(5)` — lower recency = higher score (i.e., more recent). |
| **Frequency Score** | Higher frequency of purchase → higher `f_score`. Scored using `ntile(5)`.                                |
| **Monetary Score**  | Customers who spent more receive higher `m_score`, also via `ntile(5)`.                                  |
| **Field Selection** | Retained only `CustomerID`, `r_score`, `f_score`, and `m_score`.                                         |

---

### 🎯 Why This Transformation Was Necessary

* **Enables RFM Segmentation**: Scoring simplifies raw metrics into interpretable levels of customer value.
* **Supports customer targeting**: Identifies "Champions", "At Risk", "Hibernating", and other lifecycle stages.
* **Enables modeling and visualization**: Scores can be grouped into RFM bands for easier cohort analysis.

---

### 🛠️ How It Was Done

#### 1. **Score Calculation with Window Functions**

Using `ntile(5)` to assign quintile-based ranks:

```sql
SELECT
    CustomerID,
    NTILE(5) OVER (ORDER BY Recency ASC) AS r_score,   -- Lower recency → higher score
    NTILE(5) OVER (ORDER BY Frequency DESC) AS f_score, -- Higher frequency → higher score
    NTILE(5) OVER (ORDER BY Monetary DESC) AS m_score   -- Higher monetary → higher score
FROM {{ ref('int_customer_rfm_values') }}
```

#### 2. **Scoring Logic**

* `r_score`: 5 = most recent, 1 = longest inactive
* `f_score`: 5 = most frequent, 1 = least frequent
* `m_score`: 5 = highest spender, 1 = lowest spender

---

## 🔧 Data Transformation – `int_customer_cltv_segmentation`

The `int_customer_cltv_segmentation` model classifies each customer into a **CLTV profit segment** — `"Low"`, `"Medium"`, or `"High"` — based on their **lifetime profit contribution**. This segmentation enables targeted marketing, retention strategies, and prioritization of high-value customers.

---

### ✅ What Was Transformed

| Transformation            | Description                                                                                  |
| ------------------------- | -------------------------------------------------------------------------------------------- |
| **Percentile Thresholds** | Computed the 33rd and 66th percentiles of `customer_cltv_profit` across all customers.       |
| **CLTV Segmentation**     | Assigned each customer to a `cltv_profit_segmentation` bucket using the computed thresholds. |
| **Output Fields**         | Retained only `CustomerID` and the resulting segmentation label.                             |

---

### 🎯 Why This Transformation Was Necessary

* **Simplifies customer value**: Converts continuous CLTV values into discrete, explainable tiers.
* **Enables actionability**: Empowers marketers and product teams to treat high-, medium-, and low-value customers differently.
* **Supports modeling and dashboarding**: Useful as a feature in churn prediction, lifetime value modeling, or retention dashboards.

---

### 🛠️ How It Was Done

#### 1. **Threshold Calculation**

Used a subquery (`threshold`) to calculate the 33rd and 66th percentiles:

```sql
percentile_cont(0.33) WITHIN GROUP (ORDER BY customer_cltv_profit) AS p33,
percentile_cont(0.66) WITHIN GROUP (ORDER BY customer_cltv_profit) AS p66
```

#### 2. **CLTV Segmentation Logic**

Assigned each customer to a segment based on their `customer_cltv_profit` value:

```sql
CASE  
  WHEN customer_cltv_profit < p33 THEN 'Low'
  WHEN customer_cltv_profit < p66 THEN 'Medium'
  ELSE 'High'
END AS cltv_profit_segmentation
```

#### 3. **Final Output**

Joined the threshold values using `CROSS JOIN` to apply them across the customer table:

```sql
SELECT 
  CustomerID,
  cltv_profit_segmentation
FROM int_customer_cltv
CROSS JOIN threshold
```

## 🔧 Data Transformation – `int_customer_churn_segmentation`

The `int_customer_churn_segmentation` model assigns each customer to a **churn risk group** (`Low`, `Medium`, `High`, or `Churned`) based on their **recency of activity** relative to a predefined set of thresholds.

---

### ✅ What Was Transformed

| Transformation             | Description                                                                                                                             |
| -------------------------- | --------------------------------------------------------------------------------------------------------------------------------------- |
| **Churn Group Assignment** | Used the `Recency` value and a churn flag (`IsChurned`) to classify customers into churn risk tiers.                                    |
| **Threshold Mapping**      | Applied configurable recency thresholds for `Low`, `Medium`, and `High` churn risk levels using a dbt seed file (`churn_risk_mapping`). |
| **Output Fields**          | Returned `CustomerID` and the assigned `churn_risk_group`.                                                                              |

---

### 🎯 Why This Transformation Was Necessary

* **Enables proactive retention**: Helps identify users at different levels of churn risk, even before they fully churn.
* **Supports lifecycle segmentation**: Adds an interpretable behavioral tier based on customer inactivity.
* **Feeds dashboards and targeting**: Used in marketing, lifecycle campaigns, and churn mitigation reporting.

---

### 🛠️ How It Was Done

#### 1. **Inputs**

* `int_customer_churn_flag`: provides `CustomerID` and churn flag (`IsChurned`)
* `int_customer_rfm_values`: provides `Recency` per customer
* `churn_risk_mapping` (seed file): defines cutoff thresholds for `Low`, `Medium`, and `High` risk

Example contents of `churn_risk_mapping`:

| low\_risk | medium\_risk | high\_risk |
| --------- | ------------ | ---------- |
| 30        | 60           | 90         |

#### 2. **Churn Risk Classification Logic**

```sql
CASE
  WHEN c.IsChurned = 'TRUE' THEN 'Churned'
  WHEN r.Recency <= m.low_risk THEN 'Low'
  WHEN r.Recency <= m.medium_risk THEN 'Medium'
  WHEN r.Recency <= m.high_risk THEN 'High'
END AS churn_risk_group
```

#### 3. **Final Join and Selection**

```sql
SELECT
    c.CustomerID,
    <churn risk logic>
FROM {{ ref('int_customer_churn_flag') }} c
LEFT JOIN {{ ref('int_customer_rfm_values') }} r ON r.CustomerID = c.CustomerID
CROSS JOIN {{ ref('churn_risk_mapping') }} m
```

## 🔧 Data Transformation – `int_customer_rfm_segmentation`

The `int_customer_rfm_segmentation` model assigns each customer to a **behavioral segment** based on their **RFM scores** (`r_score`, `f_score`, `m_score`). This model maps numerical RFM profiles to human-readable labels like "Champions", "At Risk", or "Hibernating" using a predefined ruleset.



 ✅ What Was Transformed

| Transformation              | Description                                                                                      |
| --------------------------- | ------------------------------------------------------------------------------------------------ |
| **RFM Segment Assignment**  | Mapped customers' individual RFM scores to predefined segment labels using a seed-based ruleset. |
| **Join with Mapping Table** | Matched RFM scores to ranges defined in `rfm_segment_mapping` (a dbt seed file).                 |
| **Output Fields**           | Returned only `CustomerID` and `rfm_segmentation` label.                                         |

---

### 🎯 Why This Transformation Was Necessary

* **Adds business-friendly segmentation**: Converts numeric scores into interpretable customer groups.
* **Enables targeting strategies**: Used for marketing campaigns, retention efforts, and upsell strategies.
* **Supports customer lifecycle modeling**: Offers insights into engagement level and purchase behavior.

---

### 🛠️ How It Was Done

#### 1. **Inputs**

* `int_customer_rfm_score`: contains customer-level RFM scores (`r_score`, `f_score`, `m_score`)
* `rfm_segment_mapping`: a seed file that defines scoring ranges and their corresponding segment labels

Example of `rfm_segment_mapping`:

| min\_r | max\_r | min\_f | max\_f | min\_m | max\_m | rfm\_segmentation |
| ------ | ------ | ------ | ------ | ------ | ------ | ----------------- |
| 4      | 5      | 4      | 5      | 4      | 5      | Champions         |
| 1      | 2      | 1      | 2      | 1      | 2      | Hibernating       |

#### 2. **Segmentation Logic**

Joined each customer’s RFM scores against the segmentation rules:

```sql
ON r_score BETWEEN seg.min_r AND seg.max_r
AND f_score BETWEEN seg.min_f AND seg.max_f
AND m_score BETWEEN seg.min_m AND seg.max_m
```

#### 3. **Final Output**

```sql
SELECT
    rfm.CustomerID,
    rfm_segmentation
FROM {{ ref('int_customer_rfm_score') }} rfm
LEFT JOIN {{ ref('rfm_segment_mapping') }} seg
  ON rfm.r_score BETWEEN seg.min_r AND seg.max_r
 AND rfm.f_score BETWEEN seg.min_f AND seg.max_f
 AND rfm.m_score BETWEEN seg.min_m AND seg.max_m
```

---







###  Business Intelligence Dashboard
Click here to view dashboard: [Dashboard](https://customer-segmentation-rfm-cltv-churn.streamlit.app/) *(*Note: Might take a minute to load*)*


---

## Data Modeling
### Final mart tables products
#### 1. OBT (One big table)
#### 2. Star Schema

1. One Big Table (OBT)
The final output is structured as a single, wide table that is fully denormalized and flattened at the customer level. 
- Used for
  - ML Models 
  - Dashboards / BI Tools 
  - Ingestion / Export 
2. One Big Table (OBT)
A normalized model with a central fact table (e.g., fct_customer_metrics) surrounded by dimension tables (e.g., dim_churn_group, dim_rfm_group)- Used for
  - Data Analysts
  - Performance in Data Warehouse 


| Layer                  | Modeling Style           | Purpose                                                                                             |
| ---------------------- | ------------------------ | --------------------------------------------------------------------------------------------------- |
| `stg_` (Staging)       | **Normalized**           | One-to-one cleanup of raw source data (no joins or heavy logic)                                     |
| `int_` (Intermediate)  | **Modular + Normalized** | Each metric calculated in isolation (AOV, DAU/MAU, CLTV, etc.) with narrow outputs                  |
| `mart_` (Final Output) | **OBT and star schema**         | Wide, joined tables with all key KPIs and labels for direct consumption by dashboards and ML models |


✅ Why This Was the Right Choice
✅ BI-Friendly: Denormalized marts reduce complexity for downstream users, ie no excessive joins (analysts, dashboards).

✅ Performance: Avoids excessive joins during query time by materializing wide, pre-joined tables.

✅ Modularity: The intermediate models are modular, reusable, and testable because they follow a normalized structure. Each model focuses on a single business metric (e.g., AOV, CLTV, Recency) and retains a clear key-to-metric relationship — typically using CustomerID as the grain. This design allows each intermediate view to be plugged into multiple downstream marts without redundancy, promoting maintainability and scalability across the analytics workflow.

### Incremental models
| **Table**                            | **Type**           | **Reason**                                                                                                                        |
| ------------------------------------ | ------------------ | --------------------------------------------------------------------------------------------------------------------------------- |
| `obt_customer_dau_mau_stickiness`    | `insert_overwrite` | Daily/Monthly activity can be recalculated at the partition level (by date), so overwriting just the new partitions is efficient. |
| `obt_customer_kpis_and_segmentation` | `merge`            | KPIs and segmentation depend on evolving customer attributes, so merging ensures updates while avoiding full rebuilds.            |
| `obt_overall_customer_kpis`          | `insert_overwrite` | Overall KPIs are aggregated by time, making partition overwrite the simplest and most efficient refresh method.                   |
| `fct_customer_metrics`               | `merge`            | Fact tables grow over time but may also update when late-arriving data comes in, so merge handles inserts + updates well.         |
| `dim_churn_group`                    | `merge (SCD 1)`    | Churn group dimension is updated with current state only (no history), so merge replaces old values efficiently.                  |
| `dim_cltv_group`                     | `merge (SCD 1)`    | CLTV group dimension also reflects only the latest grouping, requiring merge for updating rows.                                   |
| `dim_rfm_group`                      | `merge (SCD 1)`    | RFM segments can change with new activity, so merge ensures existing rows are updated in place.                                   |


##### Why Incremental Models?

Performance – Instead of recalculating the entire dataset every run, only new or updated records are processed. This saves time and compute cost.

Scalability – As data volume grows (millions or billions of rows), full refreshes become impractical. Incremental logic ensures pipelines scale.

Late-arriving data – Real-world data often lands late (e.g., delayed transactions). Incremental models (especially with merge) allow efficient backfilling without full rebuilds.

Flexibility – Different strategies (insert_overwrite vs. merge) let you choose the right trade-off: overwrite clean partitions for time-based data, or merge updates for mutable dimensions/facts.

dbt Best Practice – In modern data stacks, incremental models are considered a must-have pattern for production-grade warehouses (Snowflake, BigQuery, Redshift) to optimize pipelines.

##### What was left views 
- staging & intermediate models 


### 🏗️ Why Staging & Intermediate Models Are Views — and Marts Are Tables

- The **staging (`stg_`) and intermediate (`int_`) models are materialized as views** because:
    - They are **not queried directly by analysts or BI tools**.
    - They need to be **dynamically recalculated** each day as new data arrives.
    - They act as **reusable transformation layers** that feed multiple downstream marts.
- In contrast, the **mart models (`mart_`) are materialized as tables** because:
    - They serve as **final, analysis-ready datasets** used by analysts and visualized in dashboards.
    - They are treated as **snapshots of the data at a specific point in time**, which ensures consistency in reporting.
    - Materializing as tables improves **performance and stability** for tools consuming large, aggregated datasets.


## Data lineage:
![A description of my image](lineage.jpg)



---
🔁 Arrows = Data Dependencies
Each arrow means:

"This model depends on the model it’s pointing from."
In dbt terms: the downstream model has a ref() to the upstream model.

## Data Orchestration
- **Cron jobs** → Managed with Dagster cron jobs, which trigger dbt runs for each model on a defined schedule.  
- **Partitioning: `obt_customer_dau_mau_stickiness`** →  
  - This model is **partitioned in BigQuery by `InvoiceDate`**.  
  - Partitioning was chosen instead of standard incrementals to allow **efficient backfilling** of past dates in case a build failed 

## CI/CD
#### 🔄 CI/CD – State-Aware dbt Builds
- CI/CD pipelines are orchestrated with **GitHub Actions**.  
- dbt is configured to use **state-based testing/building** (also called **state-aware CI/CD**):  
  - Only models that have been **modified** (plus their downstream dependencies) are rebuilt.  
  - All other models are **deferred** to the existing production state.  

✅ Benefits:  
- **Faster builds** – avoids rebuilding the entire DAG.  
- **Cost-efficient** – saves compute in BigQuery/Snowflake.  
- **Safe & Scalable** – tests only changes while relying on trusted production artifacts.  

---

#### ☁️ GCP Container Registry Integration

- All Docker images used for dbt and Dagster are **built and pushed** to **Google Cloud Container Registry (GCR)**.  
- GitHub Actions workflows automatically:  
  1. **Build** the Docker image.  
  2. **Tag** with commit SHA or release version.  
  3. **Push** to GCP Container Registry.  
- This ensures a consistent, versioned image is always available for:  
  - **Local dev** (developers can pull the same image used in production).  
  - **CI pipelines** (GitHub Actions runs the same container image).  
  - **Production orchestration** (Dagster pulls the latest tested container).

## Conclusion

This project transformed raw POS transaction data into actionable customer intelligence by implementing a modular, testable dbt pipeline across staging, intermediate, and mart layers. I engineered key customer metrics — including **AOV**, **CLTV**, **RFM**, **churn**, and **engagement KPIs** — and segmented the customer base for more targeted, data-driven marketing.

Our final analytics layer enabled:

* Clear **prioritization** of high-value and at-risk customers
* Support for **personalized offers and retention strategies**
* Quantification of **customer lifetime value and churn risk**
* Generation of **marketing KPIs** like DAU, MAU, and Stickiness

This reusable architecture supports continuous updates and is built with production best practices in mind: version-controlled, documented, and testable. By leveraging tools like **dbt, Snowflake, Plotly, and Streamlit**, the pipeline enables both analysts and marketers to make faster, smarter decisions — with confidence in the data.

---

### 🔜 **Next Steps**

* Integrate campaign performance and uplift modeling
* Expand segmentation with behavioral clusters (e.g., NMF or k-means)
* Apply time-series forecasting on engagement and CLTV


